### Find original sessions of SessionID from TFDM

*import libraries*

In [1]:
import os
import sys
import glob
import sqlite3 as sl

import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
%matplotlib inline

import chime
%load_ext chime

In [2]:
pd.options.display.max_rows = 100

In [3]:
start_codes=datetime.now()

**import df from previously saved query**

In [4]:
## read .dat file if df is saved
df = pd.read_csv('./data/sql_result_onlyP002.txt')

**patient dictionary**

In [5]:
user_id_dict = {
    'TTSH-CHISELRGgLPwL3Lk6JTgBaVP4bjg==':'Test_01',
    'TTSH-CHISEL2gpumVcbJE267TKKceWbOA==':'NULL', 
    'TTSH-CHISEL0Pa/Gtc3DUqoPVREp+I/qg==':'001', 
    'TTSH-CHISEL9GsWU5zb50CfPyoLNy7qvg==':'002',
    'TTSH-CHISELamITRYYb9UymNH+3xzylgg==':'003', 
    'TTSH-CHISEL6IRO4pIMDUCpIRFbGnO26w==':'004',
    'TTSH-CHISELZSJj4EGZh0iiozqhx3Gf4g==':'005',
    'TTSH-CHISELCn5PqGNyMkKoyFYaOryw5g==':'006'
}; 


In [6]:
# replace PatientID with username
df.replace({'PatientID':user_id_dict},inplace=True)

In [7]:
df.head()

,PatientID,SessionID,GameID,TimeStampDate,TimeStampMS,Sparc,VisualTargetXCoord,VisualTargetYCoord,TargetXCoord,TargetYCoord,...,KxyGain,BxGain,ByGain,BxyGain,ByxGain,VelocityX,VelocityY,CurrentSenseLeft,CurrentSenseRight,TotalForce
0,002,NaN,0,2022-03-24 10:46:24,1916170,0.0,171.0,22.0,320.0,27.0,...,0.0,0.0,0.0,0.0,0.0,-0.001257,0.001257,-0.0432,-0.0288,0.0
1,002,TTSH-CHISELSsSRfG0v5UWvM8+yPmeYiQ==,0,2022-03-24 10:46:24,1916180,0.0,171.0,22.0,320.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,-0.0432,-0.0288,0.0
2,002,TTSH-CHISELSsSRfG0v5UWvM8+yPmeYiQ==,0,2022-03-24 10:46:24,1916190,0.0,171.0,22.0,320.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,-0.0432,-0.0288,0.0
3,002,TTSH-CHISELSsSRfG0v5UWvM8+yPmeYiQ==,0,2022-03-24 10:46:24,1916200,0.0,171.0,22.0,320.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,-0.0432,-0.0288,0.0
4,002,TTSH-CHISELSsSRfG0v5UWvM8+yPmeYiQ==,0,2022-03-24 10:46:24,1916210,0.0,171.0,22.0,320.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,-0.0432,-0.0288,0.0


In [8]:
# format date
df['date_time'] = pd.to_datetime(df['TimeStampDate'], format='%Y-%m-%d %H:%M:%S')

# remove SessionID when null
df = df[df['SessionID'].notna()]

In [9]:
# make copy
df1 = df.reset_index()

In [10]:
list_sessions = df1[df1['SessionID'].notna()]['SessionID'].unique()

In [11]:
# number of sessions in df (from TFDM)
len(list_sessions)

66

In [12]:
# extract start/end times of each session, after removal of last row
session_nr = []
start_time = []
end_time = []
for session in list_sessions:
    session_temp = df1[df1['SessionID']==session]
    
#     # drop all last index, to make life easier
#     last_index=session_temp['index'].iloc[-1]
#     df1.drop(df1[(df1['index'] == last_index)].index,axis=0,inplace=True)
    
    session_nr.append(session)
    start_time.append(session_temp['date_time'].iloc[0])
    # error in mislabelling of last data point in session
    # take last second timestamp instead
    end_time.append(session_temp['date_time'].iloc[-2])

In [13]:
# put in df
time_session = []
for row in range(0,len(session_nr)):
    time_session.append(row)
time_session = pd.DataFrame(time_session,columns=['session_nr'])   
time_session['SessionID'] = session_nr
time_session['start'] = start_time
time_session['end'] = end_time


In [14]:
time_session

,session_nr,SessionID,start,end
0,0,TTSH-CHISELSsSRfG0v5UWvM8+yPmeYiQ==,2022-03-24 10:46:24,2022-03-24 11:07:45
1,1,TTSH-CHISEL6z96a5eb70qaNCpQ6xCzow==,2022-03-24 11:26:30,2022-03-24 11:40:11
2,2,TTSH-CHISEL5ExDAtjJiEmzD2ufnUve0w==,2022-03-24 16:33:47,2022-03-24 22:48:10
3,3,TTSH-CHISELeKjzDbCwWE+4CVnBkLDBAw==,2022-03-25 09:23:46,2022-03-25 09:24:55
4,4,TTSH-CHISELk/IEgqIhBU6TdfXyNpJkWg==,2022-03-25 09:30:28,2022-03-25 10:14:56
5,5,TTSH-CHISELTgZDwLxKQ0Or7jKElwQ+gg==,2022-03-25 16:20:30,2022-03-25 17:05:37
6,6,TTSH-CHISELKPMG7CuA6kq252+6+CQfsQ==,2022-03-26 10:09:44,2022-03-26 10:53:30
7,7,TTSH-CHISELYDhQydNSg0etkwXMfqYE4g==,2022-03-26 16:26:17,2022-03-26 17:00:46
8,8,TTSH-CHISELS1kpcvQJpU255912iJ7r+w==,2022-03-26 22:16:33,2022-03-26 22:56:41
9,9,TTSH-CHISELcHiT3xi1CkC+5ihU8xqtOg==,2022-03-27 16:14:51,2022-03-27 17:06:36


In [15]:
# # save df to csv
PatientID = df1['PatientID'].unique()[0]
time_session.to_csv('./data/P' + PatientID + '_ori_sessions.csv')

In [16]:
%chime print(f'total run time = {datetime.now()-start_codes}')

total run time = 0:04:34.793454
